## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, concatenate, Reshape, InputLayer, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l1, l2, l1_l2
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 
from sklearn.svm import LinearSVC

/home/lab_services_student/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-09-25 15:24:00.959161: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 15:24:00.961462: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 15:24:00.966675: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 15:24:00.983562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting

## Data Transforming

In [2]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]

In [3]:
data

,url,status,url_length,class,num_special_chars,num_digits,num_dots,num_slash,domain,domain_length,num_digits_in_domain,subdomain,pcount,top_domain,ptld,spl,is_HTTPS,depth,subdomain_digits
0,0000111servicehelpdesk.godaddysites.com,0,39,phishing,0,7,2,0,godaddysites,12,0,0000111servicehelpdesk,2,com,1,3,0,1,7
1,000011accesswebform.godaddysites.com,0,36,phishing,0,6,2,0,godaddysites,12,0,000011accesswebform,2,com,1,3,0,1,6
2,00003.online,0,12,phishing,0,5,1,0,00003,5,5,0,1,online,1,2,0,1,0
3,0009servicedeskowa.godaddysites.com,0,35,phishing,0,4,2,0,godaddysites,12,0,0009servicedeskowa,2,com,1,3,0,1,4
4,000n38p.wcomhost.com,0,20,phishing,0,5,2,0,wcomhost,8,0,000n38p,2,com,1,3,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,zzufg.com,0,9,phishing,0,0,1,0,zzufg,5,0,0,1,com,1,2,0,1,0
798459,zzu.li,0,6,phishing,0,0,1,0,zzu,3,0,0,1,li,1,2,0,1,0
798460,zzz.co.uk,0,9,phishing,0,0,2,0,zzz,3,0,0,2,co.uk,2,3,0,1,0
798461,zzzoolight.co.za,0,16,phishing,0,0,2,0,zzzoolight,10,0,0,2,co.za,2,3,0,1,0


In [4]:
# 


In [5]:
df.url

0            0000111servicehelpdesk.godaddysites.com
1               000011accesswebform.godaddysites.com
2                                       00003.online
3                0009servicedeskowa.godaddysites.com
4                               000n38p.wcomhost.com
                             ...                    
798458                                     zzufg.com
798459                                        zzu.li
798460                                     zzz.co.uk
798461                              zzzoolight.co.za
798462    zzzoolight.co.za0-i-fdik.000webhostapp.com
Name: url, Length: 798463, dtype: object

In [6]:

label_encoder = LabelEncoder()
data['top_domain_encoded'] = label_encoder.fit_transform(data['top_domain'])
data['domain_encoded'] = label_encoder.fit_transform(data['domain'])
data['subdomain_encoded'] = label_encoder.fit_transform(data['subdomain'])
data=data.drop(columns=['url','domain','class','ptld','spl','subdomain','top_domain','pcount'])


In [7]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [8]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [9]:
c=data.subdomain_encoded.nunique()
data.subdomain_encoded.nunique()

111203

In [10]:
d=a+b+c
d

421218

In [11]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,domain_length,num_digits_in_domain,is_HTTPS,depth,subdomain_digits,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,12,0,0,1,7,225,111707,17
1,0,36,0,6,2,0,12,0,0,1,6,225,111707,18
2,0,12,0,5,1,0,5,5,0,1,0,878,10,0
3,0,35,0,4,2,0,12,0,0,1,4,225,111707,25
4,0,20,0,5,2,0,8,0,0,1,5,225,291146,29


In [12]:

y=data['status']
X=data.drop('status', axis=1)
col=X.columns
X=StandardScaler().fit_transform(X)
X=pd.DataFrame(X,columns=col)

In [13]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=54)

In [14]:
X_train.shape

(558924, 13)

In [15]:
X_test.shape

(239539, 13)

In [16]:
X_train

,url_length,num_special_chars,num_digits,num_dots,num_slash,domain_length,num_digits_in_domain,is_HTTPS,depth,subdomain_digits,top_domain_encoded,domain_encoded,subdomain_encoded
565278,-0.527616,-0.473525,-0.363395,-0.750244,-0.806292,-0.760534,-0.211449,-0.424454,-0.218199,-0.116224,-0.551873,0.310505,-0.591072
563177,-0.527616,-0.473525,-0.363395,-0.072874,-0.262065,-0.966244,-0.211449,-0.424454,0.343734,-0.116224,2.562604,0.283721,-0.591072
680861,0.312164,0.728495,-0.175586,-0.072874,0.826390,1.502273,1.682923,-0.424454,-0.780133,-0.116224,-0.551873,-1.641480,-0.591072
707757,-0.191704,-0.473525,-0.363395,-0.750244,1.914844,-0.349115,-0.211449,-0.424454,0.343734,-0.116224,-0.551873,-0.145213,-0.591072
372716,-0.143717,-0.473525,-0.363395,-0.072874,-0.262065,0.062305,-0.211449,-0.424454,0.343734,-0.116224,1.672753,-0.125565,-0.591072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224993,-0.575604,-0.473525,-0.363395,-0.072874,-0.262065,-0.349115,-0.211449,-0.424454,-0.218199,-0.116224,-0.551873,-0.733869,-0.591072
267287,-0.239692,-0.473525,-0.363395,0.604495,-0.262065,-0.349115,-0.211449,-0.424454,-0.218199,-0.116224,-0.842915,0.152235,0.867637
323343,-0.143717,-0.473525,-0.363395,-0.072874,-0.262065,-0.349115,-0.211449,-0.424454,0.343734,-0.116224,-0.551873,0.028666,0.021798
367941,0.456126,-0.173020,-0.175586,-0.072874,-0.262065,-0.966244,-0.211449,-0.424454,0.343734,-0.116224,-0.551873,1.476044,0.884194


## Traditional Models

In [17]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)
# print(classification_report(y_test, y_pred))
# trains in around 2 minutes and has 0.95 across the board

In [18]:
# vectorizer = TfidfVectorizer(analyzer='char')
# X=vectorizer.fit_transform(df.url)
# tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
# y1=df['status']
# X1=tfidf_df
# X_train1, X_test1, y_train1,y_test1 = train_test_split(X,y,test_size=0.2,random_state=504)


In [19]:
# rf = RandomForestClassifier()
# rf.fit(X_train1, y_train1)
# y_pred1 = rf.predict(X_test1)
# print(classification_report(y_test1, y_pred1))
# took 32 minutes to train on unstructured data, was less accurate for the most part 

In [20]:
# clf = svm.SVC(kernel='linear') 

# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
# ran for 5 hours and did not finish training

In [21]:
# clf = LinearSVC() 

# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
# ran for 5 minutes and did not finish training

In [22]:

# clf = SGDClassifier() 

# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
# took  4 minuted but terrible results

# Neural Networks

In [23]:
tld_num=data['top_domain_encoded'].nunique()
ld_num=data['domain_encoded'].nunique()
sub_num=data['subdomain_encoded'].nunique()



## LSTM

In [24]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)


In [25]:
Lstm  = Sequential()
Lstm.add(Embedding(a,8))
Lstm.add(Dropout(0.2))
Lstm.add(LSTM(128,return_state=False))
Lstm.add(Dense(24,activation='relu'))
Lstm.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))

In [26]:
Lstm.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [27]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyL=Lstm.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around8 minutes an epoch when split 

Epoch 1/30


17467/17467 - 280s - 16ms/step - accuracy: 0.8458 - loss: 0.3763 - precision: 0.8150 - recall: 0.9206 - val_accuracy: 0.8680 - val_loss: 0.3341 - val_precision: 0.8375 - val_recall: 0.9343
Epoch 2/30
17467/17467 - 309s - 18ms/step - accuracy: 0.8684 - loss: 0.3313 - precision: 0.8382 - recall: 0.9343 - val_accuracy: 0.8724 - val_loss: 0.3213 - val_precision: 0.8392 - val_recall: 0.9418
Epoch 3/30
17467/17467 - 311s - 18ms/step - accuracy: 0.8713 - loss: 0.3237 - precision: 0.8408 - recall: 0.9367 - val_accuracy: 0.8735 - val_loss: 0.3188 - val_precision: 0.8403 - val_recall: 0.9425
Epoch 4/30
17467/17467 - 278s - 16ms/step - accuracy: 0.8726 - loss: 0.3200 - precision: 0.8413 - recall: 0.9388 - val_accuracy: 0.8743 - val_loss: 0.3163 - val_precision: 0.8420 - val_recall: 0.9416
Epoch 5/30
17467/17467 - 312s - 18ms/step - accuracy: 0.8734 - loss: 0.3177 - precision: 0.8416 - recall: 0.9403 - val_accuracy: 0.8749 - val_loss: 0.3149 - val_precision: 0.8422 - val_recall: 0.9425
Epoch 6/30


## CNN

In [27]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)

In [28]:
cnn  = Sequential()
cnn.add(Embedding(d,8))
cnn.add(Dropout(0.2))
cnn.add(Conv1D(128, 5, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(24,activation='relu'))
cnn.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))

In [29]:
cnn.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [30]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyC=cnn.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around 6 minutes an epoch when split 

Epoch 1/30


17467/17467 - 145s - 8ms/step - accuracy: 0.9257 - loss: 0.1922 - precision: 0.9184 - recall: 0.9450 - val_accuracy: 0.9532 - val_loss: 0.1223 - val_precision: 0.9439 - val_recall: 0.9702
Epoch 2/30
17467/17467 - 143s - 8ms/step - accuracy: 0.9740 - loss: 0.0796 - precision: 0.9741 - recall: 0.9775 - val_accuracy: 0.9520 - val_loss: 0.1221 - val_precision: 0.9579 - val_recall: 0.9521
Epoch 3/30
17467/17467 - 145s - 8ms/step - accuracy: 0.9866 - loss: 0.0475 - precision: 0.9880 - recall: 0.9869 - val_accuracy: 0.9483 - val_loss: 0.1287 - val_precision: 0.9533 - val_recall: 0.9498
Epoch 4/30
17467/17467 - 149s - 9ms/step - accuracy: 0.9902 - loss: 0.0352 - precision: 0.9916 - recall: 0.9902 - val_accuracy: 0.9462 - val_loss: 0.1321 - val_precision: 0.9578 - val_recall: 0.9409
Epoch 5/30
17467/17467 - 144s - 8ms/step - accuracy: 0.9919 - loss: 0.0292 - precision: 0.9931 - recall: 0.9918 - val_accuracy: 0.9469 - val_loss: 0.1285 - val_precision: 0.9589 - val_recall: 0.9409
Epoch 5: early s